
# Module 2 - In this jupyter notebook, the seasonal T, ET, RET, NPP are calculated 

* Etape 2a - Réglage: Importer les modules/librairies, inportet les données, créer le dossier de sortie
* Etape 2b - Définition des fonctions 
* Etape 2c - Calculer la T, ET, ET de Référence (RET), ETp, NPP (Production primaire nette)

**=====================================================================================================================**

![title](img/Fig2_1.png)

**=====================================================================================================================**

## Etape 2a - Reglage

## i) Importer les modules/librairies

In [1]:
import os                                 # module d'interaction avec le système d'exploitation
import sys
import glob                               # utilisé pour retrouver des fichiers/noms de chemin correspondant à un shéma donné
import re                                 # le module re sub()  peut être utilisé pour remplacer des sous-chaines de caractères

import pandas as pd                       # pour stocker et manipuler des données tabulaires dans des lignes d'observation et des colonnes de variables
import numpy as np                        # Signifie 'Numerical Python' c'est une librairie de python utilisée pour les calculs scientifiques avec des matrices
import calendar
import datetime
from matplotlib import pyplot as plt      # est une librairie de tracage de graphiques 2D dans python 

# Changer le reperoire au lieu ou les modules sont enregistrés
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## ii) Importer les données d'entrée:
* Données de WaPOR (T, AETI-ETRI, REF, NPP), 
* Dates correspondant à la couche raster et 
* Début et fin de la saison de culture et le coefficient culture (Kc) de la culture

## * Importer les données raster (WaPOR)

In [2]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

# transpiration, évapotranspiration & interception and évapotranspiration de référence
input_folderT = os.path.join(dir_proj, dir_data, "1_L2_T_filtered") 
input_fhsT = glob.glob(input_folderT + '\*.tif')  # glob.glob retourne la liste des fichiers avec leur chemins au complet

input_folderET = os.path.join(dir_proj, dir_data, "1_L2_AETI_filtered") 
input_fhsET = glob.glob(input_folderET + '\*.tif')   

input_folderRET = os.path.join(dir_proj, dir_data, "1_L1_RET_filtered") 
input_fhsRET = glob.glob(input_folderRET + '\*.tif')

input_folderNPP = os.path.join(dir_proj, dir_data, "1_L2_NPP_filtered") 
input_fhsNPP = glob.glob(input_folderNPP + '\*.tif')

## ** Importer les dates correspondant aux couches raster de WaPOR (cube code)

![title](img/Fig2_2.jpg), ![title](img/Fig2_3.png)

* Exécuter le cellule suivante active les module de WaPOR qui necessitent la connexion internet et prend du temps pour télécharger le catalogue 
* <span style='background :lightgreen' > **exécuter la cellule un seule fois, si la dernière date dans 'time_range' est mise à jour.**
* <span style='background :pink' > En exécutant cette section pour la première fois, l'info du catalogue est enregistrée dand le repertoire de travail dans un fichier excel (e.g. df_availET.xlsx).
* <span style='background :pink' > En exécutant ce notebook pour la seconde fois vous devez sauter cette cellule, mais exécuter la prochaine(# lire les df_avial dans le fichier excel déjà enregistré sous Data) 

In [3]:
import os 
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
import WaPOR                                # l'API pour interagir avec le portail de WaPOR
WaPOR.API.version = 2

# Lire l'info du cube (dataframe-trame de données) à partir du catalogue 
cube_codeT   = 'L2_T_D' 
cube_codeET  = 'L2_AETI_D' 
cube_codeRET = 'L1_RET_D' 
cube_codeNPP = 'L2_NPP_D' 

time_range   = '2009-01-01,2019-12-31'

df_availT    = WaPOR.API.getAvailData(cube_codeT,   time_range)
df_availET   = WaPOR.API.getAvailData(cube_codeET,  time_range)
df_availRET  = WaPOR.API.getAvailData(cube_codeRET, time_range)
df_availNPP  = WaPOR.API.getAvailData(cube_codeNPP, time_range)

# Enregistrer la trame de données dans excel pour y eccéder hors connexion
output_folder = os.path.join(os.path.split(os.getcwd())[0], "Data") 

df_availT.to_excel(os.path.join(output_folder,   'df_availT.xlsx'))
df_availET.to_excel(os.path.join(output_folder,  'df_availET.xlsx'))
df_availRET.to_excel(os.path.join(output_folder, 'df_availRET.xlsx'))
df_availNPP.to_excel(os.path.join(output_folder, 'df_availNPP.xlsx'))

# Obtenir votre clé peronnelle d'API de WaPOR en vous enrégistrant en haut à droite de la page accessible ici: wapor.apps.fao.org/home/1

In [4]:
# lire les df_avial à partir du fichier excel enregistré 

time_range  = '2009-01-01,2019-12-31'

df_availT   = pd.read_excel('../data/df_availT.xlsx')
df_availET  = pd.read_excel('../data/df_availET.xlsx')
df_availRET = pd.read_excel('../data/df_availRET.xlsx')
df_availNPP = pd.read_excel('../data/df_availNPP.xlsx')

## *** Definir et importer le Début de la Saison de Culture (SOS) et la Fin de la Saison de Culture (EOS)
* Editer le début et la fin des saisons de culture **dans le fichier df_SOsEos.xlsx situé dans le dossier data**
* Vous pouvez ajouter ou supprimer des lignes en fonction du nombre de saisons 

In [5]:
df_dates = pd.read_excel('../data/df_SosEos.xlsx')
df_dates

## **** Definir et importer le Kc par mois
* Editer **months** et la valeur de **Kc** correspondant dans l'ordre du début de la saison culturale (pahse initiale)  à la fin de saison culturale  (dernière phase de la saison) **dans le fichier df_Kc.xlsx situé dans le dossier data**
* Les lignes doivent être pour les mois se situant à l'intérieur de la durée de la saison culturale 
* La figure ci-dessous montre la courbe de Kc de la canne à sucre, donnée en gise d'exemple
![title](img/Fig3_2.PNG)

In [6]:
df_kc = pd.read_excel('../data/df_Kc.xlsx')
df_kc 

## iii) Dossier de sortie: En créer un ou connecter à l'existant

In [7]:
# le repertoire du dossier de sortie
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"
output_folderT           = os.path.join(dir_proj, dir_data, "2L2_T_season") 
output_folderET          = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
output_folderRET         = os.path.join(dir_proj, dir_data, "2L1_RET_season") 
output_folderRET_month   = os.path.join(dir_proj, dir_data, "2L1_RET_month")    
output_folderETp         = os.path.join(dir_proj, dir_data, "2L1_ETp_season")    #ETp (= ETc) =Kc*REF
output_folderNPP         = os.path.join(dir_proj, dir_data, "2L2_NPP_season") 

# En créer un, si le dossier n'existe pas
if not os.path.exists(output_folderT):
    os.makedirs(output_folderT) 
if not os.path.exists(output_folderET):
    os.makedirs(output_folderET) 
if not os.path.exists(output_folderRET):
    os.makedirs(output_folderRET)  
if not os.path.exists(output_folderRET_month):
    os.makedirs(output_folderRET_month)
if not os.path.exists(output_folderETp):
    os.makedirs(output_folderETp) 
if not os.path.exists(output_folderNPP):
    os.makedirs(output_folderNPP) 

## Etape 2b - Definir la fonction - la fonction additionne les rasters entre deux dates

In [8]:
# summation des rasters entre deux dates
def SumSeason(input_fhs, saveSum, sowing_date, harvesting_date, df_avail):
    """
    Add raster files (input_fhs) between sowing_date and harvesting_date.

    IHE Delft 2019
    Authors: a.chukalla@un-ihe.org
    @author: Abebe Chukalla

    Parameters
    ----------
    input_fhs : raster file
        Files to be added.
    saveSum : folder name
        Folder name where the sum to be saved.
    sowing_date : date in yyyy-mm-dd format
        Starting date of crop growth.
    harvesting_date : date in yyyy-mm-dd format
        Harvesting date of crop.
    df_avail : cube_code of the raster
        Helps to read the date of each raster file.
        
    Returns
    -------
    Sums: array
        Seasonal, sum of the raster files.
    """
    period_dates = pd.date_range(sowing_date, harvesting_date, freq='D')  # generer les dates b/n montrant les dates de semis et récolte
    period_fhs   = []

    # collecter les rasters s'ils sont entre la date de semis et de récolte
    for in_fh in input_fhs:
        # obtenir le id du raster à partir du nom du fichier
        raster_id = os.path.split(in_fh)[-1].split('.')[0]               
        # obtenir l'infor du raster en utilisant le le id du raster
        raster_info = df_avail.loc[df_avail['raster_id']==raster_id]      # le time_code (code_temps) correspondant à l'id du raster
        # obtenir la date de début et fin du raster
        raster_startdate = raster_info['time_code'].iloc[0].split(',')[0] 
        raster_startdate = re.sub(r"[[)]", "", raster_startdate)          
        raster_enddate = raster_info['time_code'].iloc[0].split(',')[-1]  
        raster_enddate = re.sub(r"[[)]", "", raster_enddate)               
        # vérifier si le raster appartien à la periode
        if ((raster_startdate in period_dates) or (raster_enddate in period_dates)):
            period_fhs.append(in_fh)

    # Ajouter les couches entre les date de semis et de recolte
    period_fhs
    period_fh = period_fhs[0]
    Sums = 0
    for period_fh in period_fhs:
        Sum = gis.OpenAsArray(period_fh, nan_values=True)
        Sums += Sum
           
    # enregistrer la matrice au format raster, le renommer avec l'id du raster et la date de semis et de recolte
    out_fh = os.path.join(saveSum, raster_id.split('_')[1] + '_' + str(sowing_date) + '_to_' + str(harvesting_date) + '.tif')        
    gis.CreateGeoTiff(out_fh, Sums, driver, NDV, xsize, ysize, GeoT, Projection)  # enregistrer la matrice 'Sums' en tant que raster
        
    return Sums  

## Etape 2c - Calculer la T, ET, RET, ETp, NPP saisonnières

## i) Calculer la transpiration (T) saisonnière

In [9]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsT[0]  

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

length = len(df_dates)  # durée de la saison

i = 0
while i < length:  
    # calculer la valeur saisonnière et l'enrégistrer dans le dossier de sortie (output_folder)
    seasonal = SumSeason(input_fhsT, output_folderT, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availT)
   
    # calculer la moyenne et l'écart type
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    # tracer la carte raster
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='T [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Transpiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.clim(0,1500)
    plt.show ()
    
    i += 1
    ;

## ii) Calculer l'evapotratranspiration saisonnière

In [10]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsET[0]    

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

length = len(df_dates)  # durée de la saison
                
i = 0
while i < length:    
    # calculer la valeur saisonnière et l'enrégistrer dans le dossier de sortie (output_folder)
    seasonal = SumSeason(input_fhsET, output_folderET, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availET)
    
    # calculer la moyenne et l'écart type
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    # tracer la carte raster
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='ETa [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Actual evapotranspiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.clim(0,1600)
    plt.show()

    i += 1
    ;

## iii) Calculer l'Evapotranspiration de référence saisonnière (RET)

In [11]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsRET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

length = len(df_dates)  # durée de la saison

i = 0
while i < length:    
    # calculer la valeur saisonnière et l'enrégistrer dans le dossier de sortie (output_folder)
    seasonal = SumSeason(input_fhsRET, output_folderRET, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availRET)

    # calculer la moyenne et l'écart type
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    # tracer la carte raster
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='REF [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Reference evapotranspiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.show ()

    i += 1
    ;

## iv) Calculer ETp(ETc) saisonnière = Kc*RET

In [12]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsRET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

length = len(df_dates)  # durée de la saison

i = 0
while i < length:
    date_s  = df_dates.SOS[i].date()   # le début de la saison culturale (SOS)
    #ETc_total[i] = 0
    print ('Monthly ETo, ETc in mm/month and Kc[-] in ', date_s.year)
    seasonal = 0
    while date_s < df_dates.EOS[i].date():    # Itérer sur la gamme de SOS et EOS de la saison active 'i' 
        # Increment by a month
        days    = calendar.monthrange(date_s.year,date_s.month)[1] # Nombre de jours dans le mois actif
        date_e  = date_s+datetime.timedelta(days=days-1)           # Date de fin du mois actif
        
        m = date_s.strftime('%B')         # Identifier le nom du mois actif
        kc_m =df_kc.loc[df_kc["Months"] == m, 'Kc'].item()  # identifier la valeur kc corrrespondant au mois
           
        # calculer la valeur moyenne et enregistrer dans output_folder (dossier de sortie)
        ET0 = SumSeason(input_fhsRET, output_folderRET_month, date_s, date_e, df_availRET) # ajout de ET de référence dans le mois actif
        ETc = kc_m*ET0    # calculer l'ETP mensuelle(ETc)

        print (date_s.year, m, np.nanmean(ET0).round(1), np.nanmean(ETc).round(1), kc_m)
        date_s  = date_e+datetime.timedelta(days=1)   # Premier jour du prochain mois
        seasonal+= ETc
    
    # Enrgistrer la matrice saisonnière au format raster, la renommer avec le nom du raster, la date de semis et de recolte
    out_fh = os.path.join(output_folderETp, 'ETc_' + str(df_dates.SOS[i].date()) + '_to_' + str(df_dates.EOS[i].date()) + '.tif')        
    gis.CreateGeoTiff(out_fh, seasonal, driver, NDV, xsize, ysize, GeoT, Projection)  # enregistrer la matrice 'Sums' en tant que raster
    print ('The mean seasonal ETc [mm] in ', date_s.year, ' is ', np.nanmean(seasonal).round(1))

    # Tracer la carte du raster
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='ETc=Kc*ETo [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('ETc ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=12)
    plt.clim()
    plt.show ()

    i += 1
    ;

## v) Calculer la Production Primaire Nette saisonnière (NPP)

In [13]:
# collecte des Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsNPP[0]

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

length = len(df_dates)  # durée de la saison

i = 0
while i < length:
    # calculer la valeur saisonnière et l'enrégistrer dans le dossier de sortie (output_folder)
    seasonal = SumSeason(input_fhsNPP, output_folderNPP, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availNPP)

    # ccalculer la moyenne et l'écart type
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    # Tracer la carte du raster
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='NPP [gC/m2/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Net primary production [gC/m2/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.show ()
    
    i += 1 
    ;